# Behavior Analysis: Total Instances of Behavior

Press SHIFT + ENTER to run code

### USER INPUT!

In [1]:
## Define project
project_name = ''

# Select group and condition, see meta.py to see how you previously defined it for your project
selected_groups = ['', '']
selected_conditions = ['', '']

### Main Code: Create Bar Graph of Total Instances for Behavior

In [2]:
###### RUN DEPENDENCIES ######
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import sys

if not os.path.join(os.path.abspath(''), '../') in sys.path:
    sys.path.append(os.path.join(os.path.abspath(''), '../'))
    
from utils.classification import load_behaviors
from utils.meta import *
import warnings

###### Load Behaviors ######
behaviors = load_behaviors(f"../processed_dataset/{project_name}/behaviors_{project_name}.pkl")

# Define the directory path
directory_path = f"../processed_dataset/{project_name}/figures/behavior_instance-counts"
if not os.path.exists(directory_path):
    os.makedirs(directory_path)

def get_num_bouts(predict, behavior_classes):
    bout_counts = []
    bout_start_idx = np.where(np.diff(np.hstack([-1, predict])) != 0)[0]
    bout_start_label = predict[bout_start_idx]
    for b, behavior_name in enumerate(behavior_classes):
        idx_b = np.where(bout_start_label == int(b))[0]
        if len(idx_b) > 0:
            bout_counts.append(len(idx_b))
        else:
            bout_counts.append(np.NAN)
    return bout_counts

rows = len(selected_groups)
cols = len(selected_conditions)
fig, ax = plt.subplots(rows, cols, figsize=(11, 20), sharex=False, sharey=True)

# Reshape the axes array to 2D for consistent indexing
if rows == 1 and cols == 1:
    ax = np.array([[ax]])
elif rows == 1:
    ax = np.expand_dims(ax, axis=0)  # shape becomes (1, cols)
elif cols == 1:
    ax = ax.reshape(-1, 1)  # shape becomes (rows, 1)

for row in range(rows):
    # Note: the adjustment of cols within the loop (if row > 4) is not needed unless you have specific conditions.
    for col in range(cols):
        
        selected_group = selected_groups[row]
        selected_condition = selected_conditions[col]
        
        bout_counts = []
        if selected_group in behaviors and selected_condition in behaviors[selected_group]:
            file_keys = list(behaviors[selected_group][selected_condition].keys())
            
            for file_name in file_keys:
                bout_counts.append(get_num_bouts(behaviors[selected_group][selected_condition][file_name], behavior_names))
        
            with warnings.catch_warnings():
                warnings.simplefilter("ignore", category=RuntimeWarning)
                bout_mean = np.nanmean(bout_counts, axis=0)
                bout_std = np.nanstd(bout_counts, axis=0)
            
            behavior_instance_dict = {'mean_counts': bout_mean,
                                      'std_counts': bout_std,
                                      'labels': behavior_names,
                                      'colors': behavior_colors,
                                     }
            behavior_instance_df = pd.DataFrame(behavior_instance_dict)
            
            csv_filename = f"../processed_dataset/{project_name}/figures/behavior_instance-counts/behavior_instance_counts_{project_name}_{selected_group}-{selected_condition}.csv"
            behavior_instance_df.to_csv(csv_filename)

            # Plot using the properly reshaped axes array.
            behavior_instance_df.plot.barh(y='mean_counts', 
                                           x='labels', 
                                           xerr='std_counts', 
                                           color=behavior_colors, 
                                           legend=False, 
                                           ax=ax[row, col], 
                                           zorder=3)
            
            ax[row, col].set_title(f'{selected_group} - {selected_condition}')
            ax[row, col].grid(True, zorder=0)
            ax[row, col].spines['top'].set_color('#D3D3D3')
            ax[row, col].spines['right'].set_color('#D3D3D3')
            ax[row, col].spines['bottom'].set_color('#D3D3D3')
            ax[row, col].spines['left'].set_color('#D3D3D3')
            
            if row == rows-1:
                ax[row, col].set_xlabel('Instance counts / 30min')
    
fig.savefig(f"../processed_dataset/{project_name}/figures/behavior_instance-counts/behavior_counts_{project_name}.svg", 
            dpi=600, bbox_inches='tight')
            
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: '../processed_dataset/project_BK_06092025_OptoCapLeftHindpaw/behaviors_project_BK_06092025_OptoCapLeftHindpaw.pkl'

# ANALYSIS COMPLETE!

## Additional Analysis: Raw Frequency of Behavioral Bouts

In [ ]:
import warnings
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns


raw_directory_path = f"../processed_dataset/{project_name}/figures/behavior_instance-counts/behavior_instance-counts_raw"
if not os.path.exists(raw_directory_path):
    os.makedirs(raw_directory_path)

all_data = []

for selected_group in selected_groups:
    for selected_condition in selected_conditions:
        if selected_group in behaviors and selected_condition in behaviors[selected_group]:
            file_keys = list(behaviors[selected_group][selected_condition].keys())
            
            bout_counts = []
            for file_name in file_keys:
                per_file_bout_counts = get_num_bouts(behaviors[selected_group][selected_condition][file_name], behavior_names)
                bout_counts.append(per_file_bout_counts)
                
                for behavior, count in zip(behavior_names, per_file_bout_counts):
                    all_data.append({
                        'Group': selected_group,
                        'Condition': selected_condition,
                        'File': file_name,
                        'Behavior': behavior,
                        'Count': count
                    })
            
            raw_bout_counts_df = pd.DataFrame(bout_counts, columns=behavior_names, index=file_keys)

            raw_csv_filename = f"{raw_directory_path}/behavior_instance_counts_raw_{project_name}_{selected_group}-{selected_condition}.csv"
            raw_bout_counts_df.to_csv(raw_csv_filename)

            # Compute mean and std for quick check
            with warnings.catch_warnings():
                warnings.simplefilter("ignore", category=RuntimeWarning)
                bout_mean = np.nanmean(bout_counts, axis=0)
                bout_std = np.nanstd(bout_counts, axis=0)

all_data_df = pd.DataFrame(all_data)

# Summary plot
sns.set(style="whitegrid")
g = sns.catplot(
    data=all_data_df,
    x='Behavior',
    y='Count',
    hue='Behavior',
    col='Condition',
    row='Group',
    kind='box',
    height=4,
    aspect=1.2,
    palette=behavior_colors if 'behavior_colors' in globals() else 'Set2'
)

g.set_titles("{row_name} - {col_name}")
g.set_axis_labels("Behavior", "Instance counts / 30min")

# Rotate x-axis behavior labels vertically
for ax in g.axes.flat:
    ax.set_xticklabels(ax.get_xticklabels(), rotation=90)

# Save the figure
summary_figure_path = f"../processed_dataset/{project_name}/figures/behavior_instance-counts/behavior_instance-counts_raw/behavior_instance_counts_raw_summary_{project_name}.svg"
plt.savefig(summary_figure_path, dpi=300, format='svg', bbox_inches='tight')
plt.show()